In [1]:
import pandas as pd

In [2]:
#read companies
target_car=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_car')
target_og=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_OG')
target_airline=pd.read_excel('data/company_data/Data.xlsx', sheet_name='Targets_Airlines')

company_list=list(target_car['CDP_ID'].unique())+list(target_og['CDP_ID'].unique())+list(target_airline['CDP_ID'].unique())

#read ets allowance purchase data
df_ets=pd.read_csv('data/CDP/ETS_2023.csv')
df_ets = df_ets[~(df_ets['scope1_share'] == 'Question not applicable')]


df_ets['allowances_purchased']=pd.to_numeric(df_ets['allowances_purchased'], errors='coerce')

df_ets['scope1_share']=pd.to_numeric(df_ets['scope1_share'], errors='coerce')
df_ets['price']=0


In [3]:
#read avg. ETS prices
ets_prices=pd.read_excel('data/company_data/carbon_pricing.xlsx', sheet_name='Sheet1').set_index('ETS_name')

In [4]:
#fill in prices
for i in df_ets[df_ets['allowances_purchased']>0].ets_name.unique():
    df_ets.loc[df_ets['ets_name']==i, 'price']=ets_prices.loc[i, 2022]

#calculate prices
df_ets['costs']=df_ets['allowances_purchased']*df_ets['price']
df_ets['costs']=df_ets['costs'].fillna(0)


df_ets['scope1_share']=df_ets['scope1_share'].fillna(0)
df_ets.groupby('company_name')['scope1_share'].sum()

ets_costs=df_ets.groupby('CDP_ID')['costs'].sum()

/var/folders/y_/tm05xtp10jn9jjjr7109pxs00000gr/T/ipykernel_73476/2321010004.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '86.526108' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_ets.loc[df_ets['ets_name']==i, 'price']=ets_prices.loc[i, 2022]


In [5]:
#save
ets_costs.to_pickle('data/ets_costs.pkl')